In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


input_folder = 'C:/Users/Davide/workspace/SoccerData/Input/'
output_folder = 'C:/Users/Davide/workspace/SoccerData/Output/'

sns.set_style("white")

%matplotlib inline

In [2]:
df = pd.read_csv(input_folder + 'dataset.csv', index_col=0)
player = pd.read_json(input_folder+ 'players.json')
player.role = [i['code2'] for i in player.role]

df = df.merge(player[['wyId', 'lastName','role']], left_on= 'playerId',right_on= 'wyId' )

In [3]:
df.role.unique()

array(['MD', 'DF', 'FW', 'GK'], dtype=object)

In [4]:
df.head()

,playerId,foul_pct_WRT_x,matchId,matchPeriod,accurate_pass_pct_WRP,accurate_shot_pct_WRP,duel_pct_WRT,fk_corner_pct_WRT,fk_cross_pct_WRT,fk_shot_pct_WRT,...,standard_deviation,height,weight,tempo_medio_tra_eventi,ambidestro,angle_mean,angle_std,wyId,lastName,role
0,114,0.333333,2575959,1H,0.404762,NaN,0.120968,NaN,NaN,NaN,...,30.670747,186.0,78.0,59.420901,0.769231,3.311297,1.768162,114,Strootman,MD
1,114,0.142857,2575959,2H,0.275862,NaN,0.121739,NaN,NaN,NaN,...,33.559639,186.0,78.0,95.261190,0.769231,2.872542,1.789314,114,Strootman,MD
2,114,0.200000,2575976,2H,0.555556,NaN,0.104167,NaN,NaN,NaN,...,23.083842,186.0,78.0,52.515203,0.769231,2.782162,2.002138,114,Strootman,MD
3,114,0.400000,2575987,1H,0.272727,0.030303,0.101449,NaN,NaN,NaN,...,27.303354,186.0,78.0,75.670909,0.769231,3.362374,1.845300,114,Strootman,MD
4,114,0.400000,2575987,2H,0.410256,NaN,0.104000,NaN,NaN,NaN,...,22.923215,186.0,78.0,71.724029,0.769231,3.008122,1.985446,114,Strootman,MD


In [65]:
# for col in df.columns:
#     if col not in ['playerId', 'matchId','matchPeriod','lastName','wyId','role']:
#         plt.figure(figsize=(12, 5))
#         plt.title(col, fontsize = 15)
#         plt.hist(df[col].fillna(-1).values, normed=True)
#         plt.grid()

In [66]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler

x = df[df.role =='DF'].drop(['playerId', 'matchId','matchPeriod','lastName','wyId','role'],1)
x.angle_mean.fillna(x.angle_mean.mean(), inplace=True)
x.angle_std.fillna(0, inplace=True)
x.height.fillna(x.height.mean(), inplace=True)
x.weight.fillna(x.weight.mean(), inplace=True)
x.ambidestro.fillna(x.ambidestro.mean(), inplace=True)
x = x.fillna(0)


# scaler = MinMaxScaler()
# Xsc = scaler.fit_transform(x[['standard_deviation',
#                              'angle_mean', 'angle_std', 'height', 'weight', 'yellow_card',
#                              'assist', 'tempo_medio_tra_eventi','posizione_media_x', 'posizione_media_y']])

# X = np.hstack((Xsc, x.drop(['posizione_media_x', 'posizione_media_y', 'standard_deviation',
#                              'angle_mean', 'angle_std', 'height', 'weight', 'yellow_card',
#                              'assist', 'tempo_medio_tra_eventi'], axis=1)))

# X_embedded = TSNE(n_components=2, learning_rate=20).fit_transform(X)

In [67]:
# plt.figure(figsize=(10,10))
# plt.scatter(X_embedded[:,0],X_embedded[:,1])

In [68]:
example = [21001.0, 248950.0, 20495.0,
        70084.0, 25849.0, 135903.0,
       20459.0, 3300.0, 3431.0,
        130.0, 288663.0, 263432.0,
        20461.0, 92966.0, 22883.0,
        37732.0, 20438.0, 625.0,
        40726.0, 7965.0, 14903,
       20443, 3543, 21315,
       285109, 3318, 20879, 265865,
       21171, 114, 286223,
       20845, 22566, 20935,
       116269, 190167,
       3484, 21025, 21169,
       213948, 21123, 15524,
       3313, 14812, 447804,
       11063, 21077, 105334, 21385,
       89186, 20479, 122,
       206314, 3323,
       14943, 8327, 22933]

# Best feature Search 

In [5]:
prole = pd.read_csv(input_folder + 'players_with_wyrole.csv')

from itertools import product

def adjust(x):
    x =x.drop(['playerId', 'matchId','matchPeriod','lastName','wyId','role'],1)
    x.angle_mean.fillna(x.angle_mean.mean(), inplace=True)
    x.angle_std.fillna(0, inplace=True)
    x.height.fillna(x.height.mean(), inplace=True)
    x.weight.fillna(x.weight.mean(), inplace=True)
    x.ambidestro.fillna(x.ambidestro.mean(), inplace=True)
    x = x.fillna(0)
    return x

In [6]:
from numba import guvectorize, jit

@jit(["float64(float64[:],float64[:])"], nopython=True)
def cosine(a,b):
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    


@jit(["float64(float64[:],float64[:])"], nopython=True)
def euclidean(a,b):
    return np.linalg.norm(a-b)

In [7]:
@jit(["int8[:](float64[:],float64[:], float64,float64[:],float64[:])"])
def best_features(x1, x2, init_score,list_name, drop_ix= np.array([])):
    best_score = init_score
    best_ix = 0
    ix =0
    name_dropped = ''
    for ix in range(len(x1)):
        y1,y2 = np.delete(x1, ix),np.delete(x2, ix)
        score = cosine(y1,y2)
        if score > best_score:
            best_ix = ix
            best_score = score
            name_dropped = list_name[ix]
    if best_score > init_score and len(drop_ix)<10:
        list_name.remove(name_dropped)
        drop_ix = np.append(drop_ix, name_dropped)
        return best_features(np.delete(x1, best_ix),np.delete(x2, best_ix), best_score,list_name,drop_ix)
    else:
        return drop_ix

In [49]:
diocane = []

biglia = adjust(df[df.playerId ==37732]).values
veron = adjust(df[df.playerId ==625]).values

for i,j in product(biglia,veron):
    ll =['foul_pct_WRT_x', 'accurate_pass_pct_WRP', 'accurate_shot_pct_WRP',
           'duel_pct_WRT', 'fk_corner_pct_WRT', 'fk_cross_pct_WRT',
       'fk_shot_pct_WRT', 'foul_pct_WRT_y', 'interception_pct_WRT',
       'key_pass_pct_WRT', 'offside_pct_WRT', 'pass_cross_pct_WRT',
       'pass_high_pct_WRT', 'pass_simple_pct_WRT', 'pass_smart_pct_WRT',
       'shot_pct_WRT', 'assist', 'gol', 'yellow_card', 'posizione_media_x',
       'posizione_media_y', 'standard_deviation', 'height', 'weight',
       'tempo_medio_tra_eventi', 'ambidestro', 'angle_mean', 'angle_std']
    diocane.append(best_features(i,j, cosine(i,j),ll))

In [50]:
DIO = pd.DataFrame(diocane)
da_dropp = pd.Series(DIO.values.ravel()).value_counts()

In [51]:
biglia = adjust(df[df.playerId ==37732]).values
allan = adjust(df[df.playerId ==40726]).values

e_la_madonna = []


for i,j in product(biglia,allan):
    ll =['foul_pct_WRT_x', 'accurate_pass_pct_WRP', 'accurate_shot_pct_WRP',
           'duel_pct_WRT', 'fk_corner_pct_WRT', 'fk_cross_pct_WRT',
       'fk_shot_pct_WRT', 'foul_pct_WRT_y', 'interception_pct_WRT',
       'key_pass_pct_WRT', 'offside_pct_WRT', 'pass_cross_pct_WRT',
       'pass_high_pct_WRT', 'pass_simple_pct_WRT', 'pass_smart_pct_WRT',
       'shot_pct_WRT', 'assist', 'gol', 'yellow_card', 'posizione_media_x',
       'posizione_media_y', 'standard_deviation', 'height', 'weight',
       'tempo_medio_tra_eventi', 'ambidestro', 'angle_mean', 'angle_std']
    e_la_madonna.append(best_features(i,j, cosine(i,j),ll))

In [52]:
LAMADONNA = pd.DataFrame(e_la_madonna)
da_tene = pd.Series(LAMADONNA.values.ravel()).value_counts()


In [53]:
da_dropp

posizione_media_y         3104
weight                    3100
tempo_medio_tra_eventi    3093
posizione_media_x         3075
standard_deviation        3061
angle_mean                2605
foul_pct_WRT_x            1788
angle_std                 1724
accurate_pass_pct_WRP     1652
foul_pct_WRT_y            1649
interception_pct_WRT      1299
pass_high_pct_WRT         1095
shot_pct_WRT               953
yellow_card                757
pass_smart_pct_WRT         636
key_pass_pct_WRT           354
pass_simple_pct_WRT        270
fk_cross_pct_WRT           244
pass_cross_pct_WRT         242
assist                     139
fk_corner_pct_WRT          121
duel_pct_WRT               111
ambidestro                  25
height                      11
gol                          2
dtype: int64

In [54]:
da_tene

posizione_media_y         3567
posizione_media_x         3544
tempo_medio_tra_eventi    3544
weight                    3542
standard_deviation        3485
angle_mean                3053
foul_pct_WRT_x            1791
foul_pct_WRT_y            1617
accurate_pass_pct_WRP     1604
angle_std                 1581
pass_smart_pct_WRT        1515
yellow_card               1498
interception_pct_WRT      1055
shot_pct_WRT               980
pass_cross_pct_WRT         744
pass_high_pct_WRT          723
key_pass_pct_WRT           610
fk_cross_pct_WRT           280
duel_pct_WRT               274
fk_corner_pct_WRT          238
pass_simple_pct_WRT        215
assist                     178
gol                         32
height                      30
dtype: int64

In [55]:
np.setdiff1d(da_dropp.index,da_tene.index)

array(['ambidestro'], dtype=object)